# XGBoost 參數調整指南

## 參數設定理由與調整策略

### 基本參數

#### `objective: 'binary:logistic'`
- **設定理由**: 因為我們要預測 `high_elevation` (高海拔與否)，這是一個二元分類問題
- **調整建議**: 
  - 二元分類: `binary:logistic`
  - 多元分類: `multi:softprob`
  - 回歸問題: `reg:squarederror`

#### `random_state: 42`
- **設定理由**: 確保結果可重現
- **調整建議**: 任何固定整數，建議保持不變以確保實驗一致性

---

### 模型複雜度控制參數

#### `max_depth: 6`
- **設定理由**: 6是一個平衡的深度，不會太淺也不會太深
- **調整策略**:
  - **增加** (7-10): 模型更複雜，可能過擬合，適合大數據集
  - **減少** (3-5): 模型更簡單，防止過擬合，適合小數據集
  - **影響**: 控制每棵樹的複雜度

#### `min_child_weight: 1`
- **設定理由**: 預設值，適合大多數情況
- **調整策略**:
  - **增加** (3-10): 更保守，減少過擬合
  - **減少** (0.5-1): 允許更多分割，增加模型複雜度
  - **影響**: 控制葉子節點的最小樣本權重

#### `gamma: 0`
- **設定理由**: 從無正則化開始，觀察模型表現
- **調整策略**:
  - **增加** (0.1-1.0): 增加正則化，減少過擬合
  - **影響**: 節點分割需要的最小損失減少量

#### `subsample: 0.8`
- **設定理由**: 80%的樣本採樣可以防止過擬合並加速訓練
- **調整策略**:
  - **增加** (0.9-1.0): 使用更多樣本，可能過擬合
  - **減少** (0.6-0.7): 更多正則化，但可能欠擬合
  - **影響**: 每棵樹使用的樣本比例

#### `colsample_bytree: 0.8`
- **設定理由**: 80%的特徵採樣增加隨機性，防止過擬合
- **調整策略**:
  - **增加** (0.9-1.0): 使用更多特徵
  - **減少** (0.6-0.7): 增加隨機性，防止過擬合
  - **影響**: 每棵樹使用的特徵比例

---

### 學習參數

#### `learning_rate: 0.1`
- **設定理由**: 平衡的學習率，不會太快也不會太慢
- **調整策略**:
  - **增加** (0.2-0.3): 快速學習，但可能不穩定
  - **減少** (0.01-0.05): 更穩定，但需要更多樹
  - **影響**: 每棵樹的貢獻權重

#### `n_estimators: 100`
- **設定理由**: 足夠的樹數量，配合 early_stopping
- **調整策略**:
  - **增加** (200-1000): 更多樹，可能更好的性能
  - **減少** (50-100): 更快訓練，但可能性能不佳
  - **影響**: 樹的總數量

---

### 正則化參數

#### `reg_alpha: 0` (L1正則化)
- **設定理由**: 從無L1正則化開始
- **調整策略**:
  - **增加** (0.1-1.0): 特徵選擇效果，使模型更稀疏
  - **影響**: L1正則化係數，促進特徵稀疏性

#### `reg_lambda: 1` (L2正則化)
- **設定理由**: 適度的L2正則化
- **調整策略**:
  - **增加** (2-10): 更強的正則化，防止過擬合
  - **減少** (0.1-0.5): 較少正則化，允許更複雜模型
  - **影響**: L2正則化係數，平滑權重

---

### 其他重要參數

#### `scale_pos_weight: 1`
- **設定理由**: 樣本平衡時使用預設值
- **調整策略**:
  - **不平衡數據**: 設為 `負樣本數 / 正樣本數`
  - **影響**: 正負樣本的權重比，處理不平衡數據

#### `eval_metric: 'logloss'`
- **設定理由**: 分類問題的標準評估指標
- **調整建議**:
  - 分類: `logloss`, `error`, `auc`
  - 回歸: `rmse`, `mae`

---

## 調參建議順序

### 1. 粗調階段
1. 先調整 `max_depth` (3-10)
2. 調整 `min_child_weight` (1-6)
3. 調整 `learning_rate` 和 `n_estimators` 的組合

### 2. 精細調整
1. 調整 `subsample` 和 `colsample_bytree` (0.6-1.0)
2. 調整正則化參數 `reg_alpha` 和 `reg_lambda`
3. 微調 `gamma`

### 3. 最終優化
1. 根據驗證集表現選擇最佳參數組合
2. 使用較小的 `learning_rate` 和較多的 `n_estimators` 進行最終訓練

---

## 過擬合和欠擬合的判斷

### 過擬合跡象
- 訓練準確率遠高於驗證準確率
- 驗證損失開始上升

**解決方案**:
- 增加正則化 (`reg_alpha`, `reg_lambda`, `gamma`)
- 減少模型複雜度 (`max_depth`)
- 增加採樣隨機性 (`subsample`, `colsample_bytree`)

### 欠擬合跡象
- 訓練和驗證準確率都很低
- 損失下降緩慢

**解決方案**:
- 增加模型複雜度 (`max_depth`)
- 減少正則化
- 增加樹的數量 (`n_estimators`)

---

## 實用調參技巧

1. **使用交叉驗證**: 確保參數選擇的穩定性
2. **網格搜索**: 系統性地搜索參數空間
3. **早停機制**: 防止過擬合，節省訓練時間
4. **監控學習曲線**: 觀察訓練和驗證性能的變化
5. **特徵重要性**: 分析哪些特徵對預測最重要
